In [1]:
import sys
sys.path.insert(0, '../../style_generation_pipeline')

In [2]:
import pandas as pd
import json
import sklearn
import glob
import pickle
from sklearn.model_selection import train_test_split

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
from data import *

In [4]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_1'

In [5]:
raw_data = pd.read_json(path + '/raw_data.jsonl', lines=True)

In [6]:
len(raw_data)

42734

- Here we generated first a sample of 10k documents called raw_data_sample.jsonl and its corresponding training and validation authors training_authors_sample.jsonl and valid_author_sample.jsonl --> but this wasn't ideal for clustering
- We then took the whole training and validation split of all authors resulting in 15k docuents and run the style generation, clustering and building the interpretable space

### Creating a sample of the data:

In [8]:
#sample a set to generate style features from
sample_raw_data = raw_data.sample(10000)

In [10]:
output_path = path + '/raw_data_sample.jsonl'

with open(output_path, "w") as f:
    f.write(sample_raw_data.to_json(orient='records', lines=True))

In [10]:
training_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

In [80]:
output_path = path + '/training_candidates_and_queries.jsonl'
with open(output_path, "w") as f:
    f.write(training_df.to_json(orient='records', lines=True))

### Prepare authors for clustering:

In [14]:
sample_training_ds = pd.read_json(path + '/raw_data_sample.jsonl', lines=True)
sample_training_ds['authorID'] = sample_training_ds.authorIDs.apply(lambda x: x[0])

In [23]:
sample_docs_ids = sample_training_ds.documentID.unique()

In [24]:
# Cluster of the training and dev
training_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

dev_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

training_df.reset_index(inplace=True)
dev_df.reset_index(inplace=True)

training_df.to_json(path + "/training_authors.json")
dev_df.to_json(path + "/valid_authors.json")


training_df[training_df.documentID.isin(sample_docs_ids)].to_json(path + "/training_authors_sample.json")
dev_df[dev_df.documentID.isin(sample_docs_ids)].to_json(path + "/valid_authors_sample.json")

In [67]:
# all_authors = sample_training_ds.authorID.unique()
# training_authors, dev_authors = train_test_split(all_authors, test_size=0.20, random_state=123)
# training_df = sample_training_ds[sample_training_ds.authorID.isin(training_authors)]
# dev_df = sample_training_ds[sample_training_ds.authorID.isin(dev_authors)]

# training_df.to_json(path + "/training_authors_sample.json")
# dev_df.to_json(path + "/valid_authors_sample.json")

### Generated Style Features:

- After running the style generation it produces a folder `filter` under the input path. I moved it to the `explanation` folder

In [73]:
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/refined_and_aggregated_features_final.csv')
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/style_corpus.csv')
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/filtered/refined_and_aggregated_features_final.csv')
df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv')

In [74]:
df.documentID.nunique()

15796

In [75]:
df.head()

,final_attribute_name,original_attribute_name,documentID,shortend_attribute_name.v1,shortend_attribute_name.v2,aggregated_name
0,overall rhythm,The author uses a mix of short and long sentences to create a sense of rhythm and flow.,000206ff-14aa-533c-8193-2a02f438cb57,The writer varies sentence lengths for rhythm and flow.,Rewritten: Varying sentence lengths enhances the overall rhythm and flow.,Rewritten: Varying sentence lengths enhances the overall rhythm and flow.
1,diverse sentence structures,"The author employs a range of sentence structures, including simple, compound, and complex sentences.",000206ff-14aa-533c-8193-2a02f438cb57,The author uses various sentence structures in their writing.,The writer employs diverse sentence structures in their work.,The writer employs diverse sentence structures in their work.
2,better understanding,The author frequently uses subordinate clauses to provide additional information and context.,000206ff-14aa-533c-8193-2a02f438cb57,The author often incorporates additional details for clarity.,The author frequently includes extra information for better understanding.,The author frequently includes extra information for better understanding.
3,extra information,The author frequently uses subordinate clauses to provide additional information and context.,000206ff-14aa-533c-8193-2a02f438cb57,The author often incorporates additional details for clarity.,The author frequently includes extra information for better understanding.,The author frequently includes extra information for better understanding.
4,varied sentence lengths,"The author uses a variety of sentence lengths, from 5-20 words, to create visual interest on the page.",000206ff-14aa-533c-8193-2a02f438cb57,The author employs a range of sentence lengths to enhance visual appeal.,The author uses varied sentence lengths for visual impact.,The author uses varied sentence lengths for visual impact.


In [70]:
print(df.documentID.nunique())
print(df.final_attribute_name.nunique())

15796
1713


In [71]:
df['documentID'].value_counts()

da5e11a5-bebf-50bc-a00f-1e6f74534e7c    11
106acf9c-49cd-57fb-bdef-4225c682f7a0     9
60b31c1a-6241-5769-9643-411aec30ff40     9
89946966-e43e-5fc5-baba-f671e1a296b2     9
ccc21a55-44e0-5246-b1d0-12f7e9d73fb0     8
cc44c7b1-d07b-524a-9225-6d508e77cddc     8
76411bd0-65d7-54e8-836f-ff7be7b3b2e2     8
c2300c5c-e8d3-53c4-a9ef-b3a23ad77f8a     8
05d85118-eb76-50a7-888e-3f90f29e5d11     8
bbf4cabb-6a18-5450-af22-b75550777d1a     8
9e8f831f-cd77-5363-b148-6d90be5f6c2d     8
1d5a98b9-49f9-5eab-92d0-56215016b8ba     8
b8807919-c080-5d25-8a46-23049572a6cc     8
5f8a2a45-74ac-5906-b617-16b4b4be08a2     8
83defebf-15b8-5c43-a414-a0a9839df5a5     8
57a4bcc7-d938-59dc-beea-ed1700dc7f5b     8
574c448d-5b9e-51d0-a8a4-afabd92bba38     8
9029595b-340d-5dd8-b5ee-7a88dfc426eb     8
3e64c381-35eb-5eb1-9ebb-f4d15eb1b5a9     8
762e32b2-a4e0-5246-9555-b648f4ba3c2b     8
fd7c321e-1722-5890-8da7-665493cc5906     8
a4b88d4b-a786-5179-a1a0-bc8c445adff9     8
5ea23b29-b190-5537-85f1-638722ba3a52     8
8aeadf0a-a6

In [72]:
df['final_attribute_name'].value_counts()

consistent verb tense                       5367
simple sentence structures                  4408
diverse sentence structures                 4189
diverse sentence lengths                    3253
style is formal                             2900
technical language                          2409
various sentence structures                 2241
overall rhythm                              1844
complex sentence structures                 1809
various sentence types                      1686
casual language                             1513
formal tone                                 1390
style is professional                       1359
specialized language                         967
clear sentence structures                    919
better balance                               823
technical terms                              690
extra information                            679
better rhythm                                652
subordinate clauses                          613
different tenses    

In [76]:
df['final_attribute_name'].nunique()

1713

#### Merge the Gram2vec style feats to our LLM-based style feats:

In [20]:
gram2vec = json.load(open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/gram2vec_top_features.jsonl'))

In [21]:
gram2vec = {x['documentID'] : x['gram2vec_feats'] for x in gram2vec}

In [24]:
gram2vec_feats = set([x for feats in gram2vec.values() for x in feats])

In [26]:
print(len(gram2vec_feats))

638


In [27]:
len(gram2vec)

53332

In [28]:
agg_df = df.groupby('documentID').agg({'final_attribute_name': lambda x: list(x)}).reset_index()

In [29]:
agg_df['final_attribute_name'] = agg_df.apply(lambda x: x['final_attribute_name'] + gram2vec[x['documentID']], axis=1)

In [30]:
agg_df.head()

,documentID,final_attribute_name
0,000d37d2-efa7-583f-b302-f0d15f552d39,"[long sentences, complex sentences, subordinate clauses, visual interest, pos_bigrams:AUX NUM, func_words:'ll, morph_tags:Degree=Cmp, morph_tags:PronType=Dem, pos_bigrams:ADV PART, pos_bigrams:ADV VERB, pos_bigrams:AUX ADV, func_words:these, pos_bigrams:NUM NOUN, morph_tags:VerbType=Mod]"
1,001fcc11-83c6-5dd1-b4a7-17b18421193a,"[complex sentence structures, multiple clauses, vivid imagery, pos_bigrams:AUX X, func_words:his, pos_bigrams:ADP PRON, morph_tags:Gender=Masc, func_words:himself, pos_bigrams:DET PRON, func_words:themselves, morph_tags:Poss=Yes, pos_bigrams:PUNCT PART, dep_labels:poss]"
2,002adbbe-d084-5a2e-a5a8-2f7b2f34e1e1,"[simple sentence structures, long sentences, consistent verb tense, present tense, declarative sentences, limited range, func_words:again, pos_bigrams:INTJ PRON, letters:D, pos_bigrams:PRON PROPN, punctuation:!, pos_bigrams:PROPN ADV, pos_bigrams:ADP PART, pos_bigrams:ADV NOUN, pos_bigrams:PROPN ADP, func_words:been]"
3,002ae3ff-0325-5dce-b23a-dfde42aa7cb6,"[complex sentence structures, precise vocabulary, formal tone, long sentences, punctuation:!, pos_bigrams:PROPN AUX, letters:H, letters:D, pos_bigrams:PROPN ADV, pos_bigrams:PROPN NOUN, pos_bigrams:DET PROPN, func_words:just, pos_bigrams:PRON PROPN, func_words:as]"
4,002df921-62ed-52a7-a646-eb22979d5ace,"[complex sentence structures, multiple clauses, additional information, subordinate clauses, passive voice, dep_labels:csubj, func_words:most, pos_bigrams:NUM ADJ, pos_bigrams:SCONJ DET, morph_tags:Gender=Neut, pos_bigrams:ADJ DET, pos_bigrams:PUNCT SCONJ, pos_bigrams:AUX CCONJ, morph_tags:VerbForm=Ger, pos_bigrams:SCONJ ADV]"


In [31]:
joined_df = agg_df.explode(['final_attribute_name'])

In [32]:
joined_df.head()

,documentID,final_attribute_name
0,000d37d2-efa7-583f-b302-f0d15f552d39,long sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,complex sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,subordinate clauses
0,000d37d2-efa7-583f-b302-f0d15f552d39,visual interest
0,000d37d2-efa7-583f-b302-f0d15f552d39,pos_bigrams:AUX NUM


In [33]:
joined_df.to_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/gram2vec_and_llm_10k_based_style_features.csv')

In [35]:
joined_df.final_attribute_name.nunique()

857

### Consturcted Interpretable space:

In [79]:
#df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_authors.pkl')
df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_authors.pkl')

In [80]:
df.cluster_label.nunique()

157

In [81]:
#vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/interpretable_space.pkl')
vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/interpretable_space.pkl')

In [82]:
len(vector_to_style_dist)

157

In [83]:
clusters_and_top10_feats = {key : {y[0]: y[1] for y in sorted(x[1].items(), key=lambda i:-i[1])[:10]} for key, x in vector_to_style_dist.items()}

In [84]:
clusters_and_top10_feats

{-1: {'diverse sentence structures': 3896.937243485107,
  'simple sentence structures': 3525.239124886807,
  'consistent verb tense': 3254.6552654425795,
  'various sentence structures': 3056.1860685677843,
  'style is formal': 2991.7562013524357,
  'diverse sentence lengths': 2945.4538348194637,
  'technical language': 2888.517150400003,
  'complex sentence structures': 2830.079193336686,
  'overall rhythm': 2521.5402318724614,
  'casual language': 2237.7618424217476},
 0: {'technical language': 1009.8526756281261,
  'style is professional': 914.9718378792628,
  'style is formal': 910.2397054539705,
  'casual language': 820.9818080163645,
  'diverse sentence lengths': 660.5148621000728,
  'complex sentence structures': 483.2371057535076,
  'specialized language': 474.8632995820517,
  'simple sentence structures': 474.7968698254498,
  'diverse sentence structures': 463.2258508093673,
  'consistent verb tense': 400.4899182352229},
 1: {'better engagement': 6.9594618213043296,
  'differe

In [81]:
#json.dump(clusters_and_top10_feats, open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability-w-gram2vec/clusters_w_top10_feats.json', 'w'))

### Generated Explanations:

In [17]:
test_df = pd.read_json('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/test_authors_explained.json')

In [22]:
test_df[['documentID', 'documetn_style_description']].head()

,documentID,documetn_style_description
0,00491e44-990a-5fa5-8ab1-ad9e86b2edd4,"[critical thinking, complex sentence structures, figurative language, rhetorical questions, informal language, style is analytical, style is persuasive, rhetorical questions, technical language, style is authoritative, style is user, concise problem statement, analytical thought, speculative methods, technical descriptions]"
1,006dcd64-cb43-5250-bd83-182f17241557,"[figurative language, style is varied, style is vivid, diverse sentence structures, style is casual, style is vividly descriptive, active voice, concise sentences, inclusive writing style, style is descriptive, reflective tone, parallel structures, vivid descriptions, evaluation is unbiased, emotional expression]"
2,00c44d94-19ad-534d-bab7-f9a9f93fc43f,"[concise writing, style is speculative, tone is neutral, style is informative, investigative writing style, multiple perspectives, verb tenses, descriptive storytelling, style is contemplative, author is knowledgeable, diverse subjects, impactful repetition, approach is organized, technical information, author is engaging]"
3,014a02f1-c239-561a-997c-13e68f3ba5c6,"[style is diverse, proper nouns, diverse sentence structures, subordinate clauses, diverse sentence connectors, emotive language, vivid description, style is succinct, concise sentences, rhetorical questions, temporal effect, encouraging tone, proper nouns, balanced view, style is rhetorical]"
4,01842e69-0101-5395-a3bf-8baf1191f522,"[style is casual, style is vivid, tone is casual, style is straightforward, style is transitional, style is clear, present tense, consistent verb tense, consistent use, style is vivid, concise sentences, tense verbs, style is vivid, style is light, active voice]"
